In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import pickle
data = pickle.load( open( "models_save/training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

import json
with open('data/intents.json', encoding="utf8") as json_data:
    intents = json.load(json_data)

In [3]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)

    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# bag of words
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return np.array(bag)

In [5]:
# load model
model.load('models_save/model.tflearn')

INFO:tensorflow:Restoring parameters from E:\Github\chatbot-AI\AI\models_save\model.tflearn


In [6]:
# data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='1', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        return print(random.choice(i['responses']))

            results.pop(0)

In [9]:
classify('T muốn mua một cái máy tính giá rẻ')

[('office', 0.9999677)]

In [10]:
response('bố mày muốn bộ xử lý intel')

Thế hệ thứ 3 phù hợp cho những tác vụ nhẹ, lướt web, xem phim. Thế hệ thứ 5 cho những tác vụ tầm trung: gaming, 


In [18]:
response('tôi muốn máy tính windows 10, chip intel, màn hình 12 inches')

Những máy tính có sử dụng hệ điều hành windows 10: 


In [15]:
classify('tôi muốn tư vấn máy tính gaming')

[('gaming', 0.6562757), ('office', 0.34210655)]

In [12]:
response('ủa alo bố mày muốn máy tính gaming')

Cửa hàng chúng tôi đã hết hàng, bạn có muốn chọn loại máy khác không


In [18]:
classify('Bạn cho tôi hỏi thời gian cửa hàng hoạt động là mấy giờ?')

[('hours', 0.951344)]

In [19]:
response('Bạn cho tôi hỏi thời gian cửa hàng hoạt động là mấy giờ?')

7 giờ sáng - 9 giờ tối hàng ngày từ thứ 2 đến thứ 7


In [17]:
classify('cảm ơn')

[('thanks', 0.9844439)]

In [21]:
classify("Mày có the")

[('menu', 0.9995566)]

In [22]:
classify("Mày có thể giúp dc gì cho tao không?")

[('menu', 0.9661945)]

In [23]:
response("Mày có thể giúp dc gì cho tao không?")

Tôi có thể giúp bạn tư vấn, lựa chọn máy tính phù hợp với bạn.
